In [6]:
!pip install -qU langchain-anthropic 
!pip install langchain-anthropic
!pip install python-dotenv
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
import os

In [7]:
# Load environment variables.
load_dotenv()

# Store the API key in a variable.
ClaudeAPI_Key = os.getenv("ClaudeAPI_Key")

In [8]:
# Additional imports for list parser.
from langchain.output_parsers import CommaSeparatedListOutputParser

In [9]:
# Initialize the model.
chat = ChatAnthropic(temperature=0, api_key="ClaudeAPI_Key", model_name="claude-3-opus-20240229")


# Initialize the output parser.
parser = CommaSeparatedListOutputParser()

# Get the output format instructions.
instructions = parser.get_format_instructions()
print(instructions)

print()

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`



In [10]:
import os
from dotenv import load_dotenv
import anthropic
import gradio as gr

# Load environment variables and set up the Anthropic client
load_dotenv()
api_key = os.getenv("ClaudeAPI_Key")
client = anthropic.Anthropic(api_key=api_key)

def analyze_and_respond(review, sentiment_code, context=None):
    # Create the initial message
    initial_message = f'''You are an AI assistant tasked with analyzing and responding to Google reviews. You will be provided with a Google review and a binary sentiment code. Your job is to verify the sentiment, and then craft an appropriate response based on whether the review is positive or negative.

Here is the Google review you need to analyze:
<google_review>
{review}
</google_review>

The provided sentiment code is:
<sentiment_code>
{sentiment_code}
</sentiment_code>

First, carefully read the review and determine if you agree with the provided sentiment code. The code '1' indicates a positive review, while '0' indicates a negative review. 

In your analysis, consider the following:
- The overall tone of the review
- Any specific praise or complaints mentioned
- The star rating, if provided

Write your sentiment analysis inside <sentiment_analysis> tags. Include your reasoning and whether you agree or disagree with the provided sentiment code.

After your analysis, proceed based on whether the review is positive or negative:

For a positive review:
1. Express gratitude for the positive feedback
2. Highlight a specific point from the review, if applicable
3. Keep the tone warm and appreciative

For a negative review:
1. Express regret for the unsatisfactory experience
2. Acknowledge the specific issues mentioned in the review
3. If no additional context is provided, request additional context by outputting <request_context> tags.
4. If additional context is provided, craft a response that:
   a. Addresses the concerns raised
   b. Explains steps being taken to improve
   c. Invites the customer to give the business another chance
5. Keep the tone professional, empathetic, and solution-oriented

Write your final response inside <response> tags.

Remember, your goal is to maintain a positive relationship with the customer, address their concerns, and uphold the reputation of the business.'''

    if context:
        initial_message += f"\n\nAdditional context provided: {context}"

    messages = [
        {"role": "user", "content": [{"type": "text", "text": initial_message}]}
    ]

    response = client.messages.create(
        model="claude-3-5-sonnet-20240620",
        max_tokens=1000,
        temperature=0,
        messages=messages
    )

    return response.content

def process_review(review, sentiment_code):
    initial_response = analyze_and_respond(review, sentiment_code)
    
    if sentiment_code == "1":  # Positive review
        final_response = initial_response
        return initial_response, final_response, gr.update(visible=False), gr.update(visible=True)
    else:  # Negative review
        return initial_response, None, gr.update(visible=True), gr.update(visible=False)

def provide_context(review, sentiment_code, context):
    final_response = analyze_and_respond(review, sentiment_code, context)
    return final_response, gr.update(visible=True)

def regenerate_response(review, sentiment_code, context):
    new_response = analyze_and_respond(review, sentiment_code, context)
    return new_response

with gr.Blocks() as demo:
    gr.Markdown("# Google Review Analyzer and Responder")
    
    review_input = gr.Textbox(label="Google Review")
    sentiment_input = gr.Radio(["0", "1"], label="Sentiment Code (0 for negative, 1 for positive)")
    analyze_button = gr.Button("Analyze Review")
    
    initial_output = gr.Textbox(label="Initial Analysis")
    
    with gr.Group(visible=False) as context_group:
        context_input = gr.Textbox(label="Additional Context (for negative reviews)")
        context_button = gr.Button("Provide Context and Get Final Response")
    
    final_output = gr.Textbox(label="Final Response")
    
    with gr.Group(visible=False) as approval_group:
        approve_button = gr.Button("Approve Response")
        reject_button = gr.Button("Reject and Regenerate Response")
    
    analyze_button.click(
        process_review,
        inputs=[review_input, sentiment_input],
        outputs=[initial_output, final_output, context_group, approval_group]
    )
    
    context_button.click(
        provide_context,
        inputs=[review_input, sentiment_input, context_input],
        outputs=[final_output, approval_group]
    )
    
    reject_button.click(
        regenerate_response,
        inputs=[review_input, sentiment_input, context_input],
        outputs=final_output
    )

demo.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
